In [ ]:
print("hello world")

hello world


ModuleNotFoundError: No module named 'pandas'

In [5]:
import pandas as pd 

In [ ]:
pickle_path = "/Users/gulshan/Downloads/996ef_test3_data_ready_inv_20_df.pkl"
df = pd.read_pickle(pickle_path)


In [17]:
df.tail()

,id,raw_text,invoice_no,invoice_date,amount,buyer_gstin,seller_gstin,parser_type,is_edited_by_maker,is_edited_by_checker,...,is_in_seller_gstin_doclingpdf_or_paddle,all3_correct_doclingpdf_or_paddle,all4_correct_doclingpdf_or_paddle,all5_correct_doclingpdf_or_paddle,final_text,final_is_in_count,seller_id,final_input,final_output,final_instructions
7394,2428143,"SLMG BEVERAGES PRIVATE LIMITED 1052,1054,1057K...",LKUNT1226000963,2025-06-21,29840.60,09AYQPA0025M1ZE,09AAYCS4927P1Z3,1,True,False,...,0,1,1,0,"## SLMG BEVERAGES PRIVATE LIMITED\n\n1052,1054...",4,8323760198207909,This invoice is from seller id 832376019820790...,"{""invoice_no"": ""LKUNT1226000963"", ""invoice_dat...",You are an accountant looking at invoices from...
7395,2427652,"WESTERN LUBE COMPANY A/6,Gala-789,Plot-74,Shr...",30227BF25P6668,2025-06-19,7606.00,27AVTPS8566P1Z1,27AAAFW7800J1Z1,1,True,False,...,1,1,1,1,<!-- image -->\n\n## WESTERN LUBE COMPANY\n\nA...,5,3031118010341511,This invoice is from seller id 303111801034151...,"{""invoice_no"": ""30227BF25P6668"", ""invoice_date...",You are an accountant looking at invoices from...
7396,2427642,"WESTERN LUBE COMPANY A/6,Gala-789,Plot-74,Shr...",30227BF25P6830,2025-06-21,71216.00,27AVTPS8566P1Z1,27AAAFW7800J1Z1,1,True,False,...,1,1,1,1,<!-- image -->\n\n## WESTERN LUBE COMPANY\n\nA...,5,3031118010341511,This invoice is from seller id 303111801034151...,"{""invoice_no"": ""30227BF25P6830"", ""invoice_date...",You are an accountant looking at invoices from...
7397,2427636,"WESTERN LUBE COMPANY A/6,Gala-789,Plot-74,Shr...",30227BF25P6854,2025-06-21,5324.00,27AVTPS8566P1Z1,27AAAFW7800J1Z1,1,True,False,...,1,1,1,1,<!-- image -->\n\n## WESTERN LUBE COMPANY\n\nA...,5,3031118010341511,This invoice is from seller id 303111801034151...,"{""invoice_no"": ""30227BF25P6854"", ""invoice_date...",You are an accountant looking at invoices from...
7398,2428173,"Tax Invoice KAKKAD AUTO 2025-26 G.E.ROAD , NEA...",SDS/13/2025-26 SDS/14/2025-26 SDS/12/2025-26,2025-06-01,2701019.00,22AAGFK0846G1ZO,99PROGSIFAUTZZZ,3,True,False,...,0,0,0,0,"## KAKKAD AUTO 2025-26\n\nG.E.ROAD , NEAR NAGP...",2,2869718044844830,This invoice is from seller id 286971804484483...,"{""invoice_date"": ""01-06-2025"", ""buyer_gstin"": ...",You are an accountant looking at invoices from...


In [ ]:
#dynamic instruction 
    1. will ask from file 
# static instruction
    1. will ask from input field

In [5]:
import json
import pandas as pd
from difflib import SequenceMatcher

def parse_if_string(d):
    """Try to parse stringified JSON into a dictionary."""
    if isinstance(d, str):
        try:
            return json.loads(d)
        except json.JSONDecodeError:
            return {}  # or raise error
    return d

def normalize(value):
    if isinstance(value, float):
        value = round(value)
    return str(value).replace(" ", "").lower().strip()

def is_partial_match(val1, val2, threshold=0.9):
    val1, val2 = normalize(val1), normalize(val2)
    return SequenceMatcher(None, val1, val2).ratio() >= threshold

def evaluate_sample(sample_id, output, model_response):
    results = []
    exact_matches = 0
    partial_matches = 0
    total_fields = len(output)

    for field in output:
        gt = output[field]
        pred = model_response.get(field)

        norm_gt = normalize(gt)
        norm_pred = normalize(pred)

        if norm_gt == norm_pred:
            status = "✅ Correct"
            exact_matches += 1
        elif is_partial_match(gt, pred):
            status = "⚠️ Partial"
            partial_matches += 1
        else:
            status = "❌ Incorrect"

        results.append({
            "sample_id": sample_id,
            "field": field,
            "expected": gt,
            "predicted": pred,
            "status": status
        })

    strict_accuracy = exact_matches / total_fields
    lenient_accuracy = (exact_matches + 0.5 * partial_matches) / total_fields

    return results, strict_accuracy, lenient_accuracy

def evaluate_file(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    all_results = []
    strict_total = 0
    lenient_total = 0
    sample_count = 0

    for i, item in enumerate(data):
        output = parse_if_string(item.get("output", {}))
        model_response = parse_if_string(item.get("model_response", {}))

        if not output or not isinstance(output, dict):
            print(f"⚠️ Skipping sample {i+1}: empty or invalid output")
            continue

        results, strict_acc, lenient_acc = evaluate_sample(i + 1, output, model_response)

        all_results.extend(results)
        strict_total += strict_acc
        lenient_total += lenient_acc
        sample_count += 1

    if sample_count == 0:
        print("❌ No valid samples found.")
        return pd.DataFrame(), {}

    df = pd.DataFrame(all_results)

    overall_report = {
        "total_samples": sample_count,
        "average_strict_accuracy": round(strict_total / sample_count, 3),
        "average_lenient_accuracy": round(lenient_total / sample_count, 3)
    }

    return df, overall_report


# ======= USAGE =======
# Replace 'your_file.json' with your JSON file path
json_file_path = 'finvix.json'
df_report, summary = evaluate_file(json_file_path)

print("\n📋 Field-Level Report:")
print(df_report)

print("\n📊 Overall Summary:")
for k, v in summary.items():
    print(f"{k}: {v}")

# Optional: export report to CSV
# df_report.to_csv("field_level_accuracy_report.csv", index=False)


⚠️ Skipping sample 126: empty or invalid output
⚠️ Skipping sample 263: empty or invalid output
⚠️ Skipping sample 439: empty or invalid output

📋 Field-Level Report:
       sample_id           field         expected        predicted  \
0              1  invoice_number              H 3               H3   
1              1    invoice_date       2024-07-17       2024-07-17   
2              1  invoice_amount           135756         135755.0   
3              1     buyer_gstin  06AMCPK6898C1ZW  09AABFJ3994L1ZC   
4              1    seller_gstin  06AVQPA8731K2Z8  09ALPPS4437P1ZN   
...          ...             ...              ...              ...   
17430       3490  invoice_number     25-26/LT-600           LT-600   
17431       3490    invoice_date       2025-05-10       2025-05-10   
17432       3490  invoice_amount            55478          55478.0   
17433       3490     buyer_gstin  03IAEPK0138P1ZL  03IAEPK0138P1ZL   
17434       3490    seller_gstin  03BKKPS3920A1ZK  03BKKPS3920A

In [7]:
import json
import pandas as pd
from difflib import SequenceMatcher
from collections import defaultdict

# ---------- Utility Functions ----------
def normalize(value):
    """Normalize values for accurate comparison."""
    try:
        # Try converting to float first
        float_val = float(value)
        if float_val.is_integer():
            return str(int(float_val))  # Convert 135755.0 → "135755"
        else:
            return str(round(float_val, 2))  # Round float to 2 decimal places
    except (ValueError, TypeError):
        # If not a number, treat as string
        return str(value).replace(" ", "").lower().strip()

def is_partial_match(val1, val2, threshold=0.9):
    """Check for close match using similarity threshold."""
    val1, val2 = normalize(val1), normalize(val2)
    return SequenceMatcher(None, val1, val2).ratio() >= threshold

def parse_if_string(d):
    """Convert JSON string to dict if needed."""
    if isinstance(d, str):
        try:
            return json.loads(d)
        except json.JSONDecodeError:
            return {}
    return d

# ---------- Evaluation per Sample ----------
def evaluate_sample(sample_id, output, model_response):
    results = []
    exact_matches = 0
    partial_matches = 0
    total_fields = len(output)

    for field in output:
        gt = output[field]
        pred = model_response.get(field)

        norm_gt = normalize(gt)
        norm_pred = normalize(pred)

        if norm_gt == norm_pred:
            status = "✅ Correct"
            exact_matches += 1
        elif is_partial_match(gt, pred):
            status = "⚠️ Partial"
            partial_matches += 1
        else:
            status = "❌ Incorrect"

        results.append({
            "sample_id": sample_id,
            "field": field,
            "expected": gt,
            "predicted": pred,
            "status": status
        })

    strict_accuracy = exact_matches / total_fields
    lenient_accuracy = (exact_matches + 0.5 * partial_matches) / total_fields

    return results, strict_accuracy, lenient_accuracy

# ---------- Full File Evaluation ----------
def evaluate_file(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    all_results = []
    strict_total = 0
    lenient_total = 0
    sample_count = 0

    for i, item in enumerate(data):
        output = parse_if_string(item.get("output", {}))
        model_response = parse_if_string(item.get("model_response", {}))

        if not output or not isinstance(output, dict):
            print(f"⚠️ Skipping sample {i+1}: empty or invalid output")
            continue

        results, strict_acc, lenient_acc = evaluate_sample(i + 1, output, model_response)

        all_results.extend(results)
        strict_total += strict_acc
        lenient_total += lenient_acc
        sample_count += 1

    if sample_count == 0:
        print("❌ No valid samples found.")
        return pd.DataFrame(), {}

    df = pd.DataFrame(all_results)

    overall_report = {
        "total_samples": sample_count,
        "average_strict_accuracy": round(strict_total / sample_count, 3),
        "average_lenient_accuracy": round(lenient_total / sample_count, 3)
    }

    return df, overall_report

# ---------- Field-wise Accuracy ----------
def compute_fieldwise_accuracy(report_df):
    field_stats = defaultdict(lambda: {"total": 0, "exact": 0, "partial": 0})

    for _, row in report_df.iterrows():
        field = row["field"]
        field_stats[field]["total"] += 1

        if row["status"] == "✅ Correct":
            field_stats[field]["exact"] += 1
        elif row["status"] == "⚠️ Partial":
            field_stats[field]["partial"] += 1

    field_accuracy_report = []
    for field, stats in field_stats.items():
        total = stats["total"]
        exact = stats["exact"]
        partial = stats["partial"]
        strict = exact / total
        lenient = (exact + 0.5 * partial) / total

        field_accuracy_report.append({
            "field": field,
            "total": total,
            "exact_matches": exact,
            "partial_matches": partial,
            "strict_accuracy_%": round(strict * 100, 2),
            "lenient_accuracy_%": round(lenient * 100, 2)
        })

    return pd.DataFrame(field_accuracy_report)

# ---------- Main Script ----------
if __name__ == "__main__":
    json_file_path = 'finvix.json'  # ✅ Replace with your file

    df_report, summary = evaluate_file(json_file_path)

    if not df_report.empty:
        print("\n📋 Field-Level Result Table:")
        print(df_report.to_string(index=False))

        print("\n📊 Overall Accuracy Summary:")
        for k, v in summary.items():
            print(f"{k}: {v}")

        print("\n📈 Per-Field Accuracy:")
        fieldwise_df = compute_fieldwise_accuracy(df_report)
        print(fieldwise_df.to_string(index=False))

        # Optional: Export to CSV
        # df_report.to_csv("field_level_results.csv", index=False)
        # fieldwise_df.to_csv("fieldwise_accuracy_summary.csv", index=False)


⚠️ Skipping sample 126: empty or invalid output
⚠️ Skipping sample 263: empty or invalid output
⚠️ Skipping sample 439: empty or invalid output

📋 Field-Level Result Table:
 sample_id          field                                                                   expected                                     predicted      status
         1 invoice_number                                                                        H 3                                            H3   ✅ Correct
         1   invoice_date                                                                 2024-07-17                                    2024-07-17   ✅ Correct
         1 invoice_amount                                                                     135756                                      135755.0 ❌ Incorrect
         1    buyer_gstin                                                            06AMCPK6898C1ZW                               09AABFJ3994L1ZC ❌ Incorrect
         1   seller_gstin       

In [8]:
import json
import pandas as pd
from difflib import SequenceMatcher
from collections import defaultdict

# ---------- Utility Functions ----------
def normalize(value):
    """Normalize values for accurate comparison."""
    try:
        # Try converting to float first
        float_val = float(value)
        if float_val.is_integer():
            return str(int(float_val))  # Convert 135755.0 → "135755"
        else:
            return str(round(float_val, 2))  # Round float to 2 decimal places
    except (ValueError, TypeError):
        # If not a number, treat as string
        return str(value).replace(" ", "").lower().strip()

def is_partial_match(val1, val2, threshold=0.9):
    """Check for close match using similarity threshold."""
    val1, val2 = normalize(val1), normalize(val2)
    return SequenceMatcher(None, val1, val2).ratio() >= threshold

def parse_if_string(d):
    """Convert JSON string to dict if needed."""
    if isinstance(d, str):
        try:
            return json.loads(d)
        except json.JSONDecodeError:
            return {}
    return d

# ---------- Evaluation per Sample ----------
def evaluate_sample(sample_id, output, model_response):
    results = []
    exact_matches = 0
    partial_matches = 0
    total_fields = len(output)

    for field in output:
        gt = output[field]
        pred = model_response.get(field)

        norm_gt = normalize(gt)
        norm_pred = normalize(pred)

        if norm_gt == norm_pred:
            status = "✅ Correct"
            exact_matches += 1
        elif is_partial_match(gt, pred):
            status = "⚠️ Partial"
            partial_matches += 1
        else:
            status = "❌ Incorrect"

        results.append({
            "sample_id": sample_id,
            "field": field,
            "expected": gt,
            "predicted": pred,
            "status": status
        })

    strict_accuracy = exact_matches / total_fields
    lenient_accuracy = (exact_matches + 0.5 * partial_matches) / total_fields

    return results, strict_accuracy, lenient_accuracy

# ---------- Full File Evaluation ----------
def evaluate_file(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    all_results = []
    strict_total = 0
    lenient_total = 0
    sample_count = 0

    for i, item in enumerate(data):
        output = parse_if_string(item.get("output", {}))
        model_response = parse_if_string(item.get("model_response", {}))

        if not output or not isinstance(output, dict):
            print(f"⚠️ Skipping sample {i+1}: empty or invalid output")
            continue

        results, strict_acc, lenient_acc = evaluate_sample(i + 1, output, model_response)

        all_results.extend(results)
        strict_total += strict_acc
        lenient_total += lenient_acc
        sample_count += 1

    if sample_count == 0:
        print("❌ No valid samples found.")
        return pd.DataFrame(), {}

    df = pd.DataFrame(all_results)

    overall_report = {
        "total_samples": sample_count,
        "average_strict_accuracy": round(strict_total / sample_count, 3),
        "average_lenient_accuracy": round(lenient_total / sample_count, 3)
    }

    return df, overall_report

# ---------- Field-wise Accuracy ----------
def compute_fieldwise_accuracy(report_df):
    field_stats = defaultdict(lambda: {"total": 0, "exact": 0, "partial": 0})

    for _, row in report_df.iterrows():
        field = row["field"]
        field_stats[field]["total"] += 1

        if row["status"] == "✅ Correct":
            field_stats[field]["exact"] += 1
        elif row["status"] == "⚠️ Partial":
            field_stats[field]["partial"] += 1

    field_accuracy_report = []
    for field, stats in field_stats.items():
        total = stats["total"]
        exact = stats["exact"]
        partial = stats["partial"]
        strict = exact / total
        lenient = (exact + 0.5 * partial) / total

        field_accuracy_report.append({
            "field": field,
            "total": total,
            "exact_matches": exact,
            "partial_matches": partial,
            "strict_accuracy_%": round(strict * 100, 2),
            "lenient_accuracy_%": round(lenient * 100, 2)
        })

    return pd.DataFrame(field_accuracy_report)

# ---------- Main Script ----------
if __name__ == "__main__":
    json_file_path = 'finvix_7000.json'  # ✅ Replace with your file

    df_report, summary = evaluate_file(json_file_path)

    if not df_report.empty:
        print("\n📋 Field-Level Result Table:")
        print(df_report.to_string(index=False))

        print("\n📊 Overall Accuracy Summary:")
        for k, v in summary.items():
            print(f"{k}: {v}")

        print("\n📈 Per-Field Accuracy:")
        fieldwise_df = compute_fieldwise_accuracy(df_report)
        print(fieldwise_df.to_string(index=False))

        # Optional: Export to CSV
        # df_report.to_csv("field_level_results.csv", index=False)
        # fieldwise_df.to_csv("fieldwise_accuracy_summary.csv", index=False)


⚠️ Skipping sample 1: empty or invalid output
⚠️ Skipping sample 2: empty or invalid output
⚠️ Skipping sample 3: empty or invalid output
⚠️ Skipping sample 4: empty or invalid output
⚠️ Skipping sample 5: empty or invalid output
⚠️ Skipping sample 6: empty or invalid output
⚠️ Skipping sample 7: empty or invalid output
⚠️ Skipping sample 8: empty or invalid output
⚠️ Skipping sample 9: empty or invalid output
⚠️ Skipping sample 10: empty or invalid output
⚠️ Skipping sample 11: empty or invalid output
⚠️ Skipping sample 12: empty or invalid output
⚠️ Skipping sample 13: empty or invalid output
⚠️ Skipping sample 14: empty or invalid output
⚠️ Skipping sample 15: empty or invalid output
⚠️ Skipping sample 16: empty or invalid output
⚠️ Skipping sample 17: empty or invalid output
⚠️ Skipping sample 18: empty or invalid output
⚠️ Skipping sample 19: empty or invalid output
⚠️ Skipping sample 20: empty or invalid output
⚠️ Skipping sample 21: empty or invalid output
⚠️ Skipping sample 22: